In [0]:
# ==========================================
# ✈️ EDA FOR SILVER FLIGHT DATA (PRE-ML)
# ==========================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# Spark session (if not already running)
spark = SparkSession.builder.getOrCreate()

# Load Silver table
print("📥 Loading Silver Table...")
df_silver = spark.table("default.silver_flights_processed")

print(f"✅ Loaded Silver Table with {df_silver.count():,} records and {len(df_silver.columns)} columns")
df_silver.printSchema()

In [0]:
# ==========================================
# 🧩 FEATURE ENGINEERING FOR EDA
# ==========================================

df_silver = df_silver.withColumn(
    "arrival_status",
    when(col("arrival_delay").isNull(), "Cancelled/Diverted")
    .when(col("arrival_delay") >= 15, "Delayed")
    .when(col("arrival_delay") < 0, "Early")
    .otherwise("On-Time")
)

# Show sample of new column
print("Sample of 'arrival_delay' and 'arrival_status':")
df_silver.select("arrival_delay", "arrival_status").show(10)

# Summary of category counts
print("\nCounts per arrival_status:")
df_silver.groupBy("arrival_status").count().show()

In [0]:
# ==========================================
# 📊 CONVERT TO PANDAS FOR VISUALIZATION
# ==========================================

df = df_silver.toPandas()
sns.set(style="whitegrid", palette="Set2", font_scale=1.1)

print("✅ Converted to Pandas for visualization")
display(df.head(5))


In [0]:
# ==========================================
# ✅ SUMMARY STATISTICS
# ==========================================

print("\n🧭 Key Insights:")
print("""
• 'arrival_status' shows the proportion of Delayed, On-Time, and Early flights.
• Airline flight counts indicate which carriers dominate the dataset.
• Monthly delay trends highlight seasonal congestion effects.
• Correlation heatmap reveals relationships (e.g., between delay and time variables).
""")

In [0]:
# ==========================================
# 🎯 DISTRIBUTION OF ARRIVAL STATUS
# ==========================================
plt.figure(figsize=(8, 5))
order = df['arrival_status'].value_counts().index
sns.countplot(data=df, x='arrival_status', order=order, palette="Set2")
plt.title("Distribution of Arrival Status")
plt.xlabel("Arrival Status")
plt.ylabel("Number of Flights")
plt.xticks()
plt.tight_layout()
plt.show()


In [0]:
# ==========================================
# 🏢 DISTRIBUTION OF AIRLINES
# ==========================================
plt.figure(figsize=(10, 6))
order = df['airline_code'].value_counts().index
sns.countplot(y='airline_code', data=df, order=order, palette="viridis")
plt.title("Number of Flights by Airline")
plt.xlabel("Number of Flights")
plt.ylabel("Airline Code")
plt.tight_layout()
plt.show()


In [0]:
# ==========================================
# 🌍 TOP 20 DESTINATION AIRPORTS
# ==========================================
top_dest = df['destination_airport_code'].value_counts().head(20)
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=top_dest.values, y=top_dest.index, palette="Blues_r")

for container in ax.containers:
    ax.bar_label(container, fmt='%d', label_type='edge', padding=3, fontsize=10, color='black')

plt.title("Top 20 Destination Airports")
plt.xlabel("Number of Arrivals")
plt.ylabel("Destination Airport Code")
plt.tight_layout()
plt.show()

In [0]:
# ==========================================
# ⏱️ DISTRIBUTION OF ARRIVAL DELAY (Trimmed)
# ==========================================
plt.figure(figsize=(8, 5))
filtered = df[df['arrival_delay'].between(-50, 250)]
sns.histplot(filtered['arrival_delay'], bins=60, kde=True, color='steelblue')
plt.title("Distribution of Arrival Delay (-50 → 250 minutes)")
plt.xlabel("Delay (minutes)")
plt.xlim(-50, 250)
plt.tight_layout()
plt.show()


In [0]:
# ==========================================
# 🗓️ MONTHLY AVERAGE ARRIVAL DELAY BY YEAR
# ==========================================
monthly_delay = (
    df.groupby(["flight_year", "flight_month"])["arrival_delay"]
    .mean()
    .reset_index()
    .dropna()
)

years = monthly_delay["flight_year"].unique()
palette = sns.color_palette("tab10", n_colors=len(years))

plt.figure(figsize=(12, 6))
sns.lineplot(
    data=monthly_delay,
    x="flight_month",
    y="arrival_delay",
    hue="flight_year",
    marker="o",
    palette=palette,
    linewidth=2.2
)
plt.title("Average Monthly Arrival Delay by Year")
plt.xlabel("Month")
plt.ylabel("Average Arrival Delay (minutes)")
plt.legend(title="Year", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [0]:
# ==========================================
# 📈 CORRELATION HEATMAP (NUMERIC FEATURES)
# ==========================================
numeric_cols = df.select_dtypes(include=[np.number]).columns
corr = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr, cmap="coolwarm", center=0, annot=False)
plt.title("Correlation Heatmap of Numeric Columns")
plt.tight_layout()
plt.show()